# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [7]:
# Create our features
X = pd.get_dummies(df.drop(columns='loan_status', axis=1),drop_first=True)

# Create our target
y = df.loan_status

In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [12]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=1,
    stratify=y
)

X_train.shape

(58152, 9)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = standard_scaler.fit(X_train)

In [15]:
# Scale the training and testing data
X_train_scaler = X_scaler.transform(X_train)
X_test_scaler = X_scaler.transform(X_test)

# Simple Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [56]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8599    0.9136    0.9950    0.8860    0.9535    0.9017       625
   low_risk     0.9971    0.9950    0.9136    0.9961    0.9535    0.9165     18759

avg / total     0.9927    0.9924    0.9162    0.9925    0.9535    0.9160     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [22]:
# Resample the training data with the RandomOversampler
naive_oversampling = RandomOverSampler(random_state=1)
X_resampled, y_resampled = naive_oversampling.fit_resample(X_train_scaler,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [23]:
# Train the Logistic Regression model using the resampled data
logistic_regression_model = LogisticRegression(solver='lbfgs', random_state=1)
logistic_regression_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
predicted_score = logistic_regression_model.predict(X_test_scaler)
balanced_accuracy_score(y_test, predicted_score)

0.9946414201183431

In [26]:
# Display the confusion matrix
confusion_matrix(y_test, predicted_score)

array([[  622,     3],
       [  111, 18648]], dtype=int64)

In [52]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predicted_score, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8486    0.9952    0.9941    0.9161    0.9946    0.9894       625
   low_risk     0.9998    0.9941    0.9952    0.9970    0.9946    0.9892     18759

avg / total     0.9950    0.9941    0.9952    0.9943    0.9946    0.9892     19384



### SMOTE Oversampling

In [29]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(
    random_state=1,
    sampling_strategy=1
).fit_resample(X_train_scaler,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [32]:
# Train the Logistic Regression model using the resampled data
smote_resampled = LogisticRegression(
    solver='lbfgs',
    random_state=1)
smote_resampled.fit(
    X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculated the balanced accuracy score
predicted_smote_score = smote_resampled.predict(X_test_scaler)
balanced_accuracy_score(y_test,predicted_smote_score)

0.9946680739911509

In [34]:
# Display the confusion matrix
confusion_matrix(y_test,predicted_smote_score)

array([[  622,     3],
       [  110, 18649]], dtype=int64)

In [53]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predicted_score, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8486    0.9952    0.9941    0.9161    0.9946    0.9894       625
   low_risk     0.9998    0.9941    0.9952    0.9970    0.9946    0.9892     18759

avg / total     0.9950    0.9941    0.9952    0.9943    0.9946    0.9892     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [37]:
# Resample the data using the ClusterCentroids resampler
X_resampled, y_resampled = ClusterCentroids(
    random_state=1,
    sampling_strategy=1
).fit_resample(X_train_scaler,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [38]:
# Train the Logistic Regression model using the resampled data
cluster_resampled = LogisticRegression(solver='lbfgs', random_state=1)
cluster_resampled.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [39]:
# Calculate the balanced accuracy score
predicted_cluster_score = cluster_resampled.predict(X_test_scaler)
balanced_accuracy_score(y_test,predicted_cluster_score)

0.9932813049736127

In [40]:
# Display the confusion matrix
confusion_matrix(y_test,predicted_cluster_score)

array([[  620,     5],
       [  102, 18657]], dtype=int64)

In [54]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,predicted_cluster_score,digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8587    0.9920    0.9946    0.9206    0.9933    0.9864       625
   low_risk     0.9997    0.9946    0.9920    0.9971    0.9933    0.9869     18759

avg / total     0.9952    0.9945    0.9921    0.9947    0.9933    0.9868     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [47]:
# Resample the training data with SMOTEENN
X_resampled,y_resampled = SMOTEENN(
    random_state=1,
    sampling_strategy=1
).fit_resample(X_train_scaler,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 55636, 'low_risk': 55936})

In [48]:
# Train the Logistic Regression model using the resampled data
smoteenn_resampled = LogisticRegression(solver='lbfgs', random_state=1)
smote_resampled.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [49]:
# Calculate the balanced accuracy score
predicted_smoteenn_score = smote_resampled.predict(X_test_scaler)
balanced_accuracy_score(y_test,predicted_smoteenn_score)

0.9946680739911509

In [50]:
# Display the confusion matrix
confusion_matrix(y_test, predicted_smoteenn_score)

array([[  622,     3],
       [  110, 18649]], dtype=int64)

In [55]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,predicted_smoteenn_score,digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8497    0.9952    0.9941    0.9167    0.9947    0.9895       625
   low_risk     0.9998    0.9941    0.9952    0.9970    0.9947    0.9893     18759

avg / total     0.9950    0.9942    0.9952    0.9944    0.9947    0.9893     19384



# Final Questions

1. Which model had the best balanced accuracy score?
    
    Both of the smote and smoteenn models have the best balanced accuracy scores.
    * smote oversampling at 0.9946680739911509
    * smoteenn at 0.9946680739911509

2. Which model had the best recall score?

    The ClusterCentroid had the best recall score at 0.9945.

3. Which model had the best geometric mean score?

    The smoteenn had the best geometric mean score at 0.9947.
